# Predicting Usage of a Video Game Research Server
Student Name: Vitor Han 

Student Number: 53497632

## Introduction

In collaboration with a UBC Computer Science research group, this project explores behavioral patterns of players on a Minecraft research server. The server records each player’s activity to understand player engagement, data contribution, and predict demand. Accurately predicting these factors helps the research team allocate server resources, target recruitment strategies, and improve player experience.

I selected Question 1 of the broad questions and use it to formulate a specific question using some of the variables in the dataset.

**Question 1:**  Can a player's gender predict their likelihood of subscribing to a game-related newsletter, and does this relationship differ between novice ("Beginner") and experienced ("Pro") players?


**Link to github repository:** https://github.com/vitxrlee/dsci-100-2025SS1-project



## Data Description 

The dataset consists of two files:

- players.csv: Contains demographic and gameplay-related features for each player (e.g., player ID, total play time, play frequency, and whether they subscribed to the newsletter).

- sessions.csv: Includes logs of each play session per player (e.g., session start/end, actions performed, duration).

### Summary of dataset:

**sessions.csv (each row represents one gameplay session and includes):**

- hashedEmail: useless in our project

- start_time: The human-readable start time of the session.

- end_time: The human-readable end time of the session.

- original_start_time: Start time in Unix timestamp format.

- original_end_time: End time in Unix timestamp format.


**players.csv (each row in this dataset indicates an individual player):**

- experience: Self-reported gaming experience, categorized as Beginner, Amateur, Regular, Veteran, or Pro.

- subscribe: Indicating whether the player subscribed to the server’s content or notifications.

- hashedEmail: A pseudonymized identifier for each player.

- played_hours: Total number of hours the player has played on the server.

- name: The first name of the player.

- gender: Gender identity (Male, Female, Non-binary).

- age: The player’s self-reported age (integer).



## Methods & Results



In [ ]:
library(tidyverse)
library(rsample)
library(tidymodels)

# Download data
player_url <- "https://raw.githubusercontent.com/vitxrlee/dsci-100-2025SS1-project/refs/heads/main/players.csv"  
session_url <- "https://raw.githubusercontent.com/vitxrlee/dsci-100-2025SS1-project/refs/heads/main/sessions.csv"

# Save locally
download.file(player_url, destfile = "players.csv")
download.file(session_url, destfile = "sessions.csv")

# Read data
players <- read_csv("players.csv") 
sessions <- read_csv("sessions.csv") 

# Simplify gender variable
players <- players |> 
  mutate(gender_simple = ifelse(
    gender == "Male", "Male",
    ifelse(gender == "Female", "Female", "Other"))
  )

# Filter only Beginner and Pro players
players_filtered <- players |> 
  filter(experience %in% c("Beginner", "Pro")) |> 
  mutate(
    subscribe = as.factor(subscribe),
    experience = factor(experience, levels = c("Beginner", "Pro")),
    gender_simple = as.factor(gender_simple)
  )

# Table of gender & subscribe
gender_experience_subscribe <- players_filtered |> 
  group_by(gender_simple, experience, subscribe) |> 
  summarize(count = n(), .groups = "drop")

# Bar plot for each group
ggplot(gender_experience_subscribe, aes(x = subscribe, y = count, fill = gender_simple)) +
  geom_bar(stat = "identity") +
  facet_wrap(~experience) +
  labs(
    title = "Newsletter Subscription by Gender and Experience",
    x = "Subscription Status", 
    y = "Player Count", 
    fill = "Gender"
  )



In [ ]:
# Logistic regression using tidymodels
set.seed(123)
data_split <- initial_split(players_filtered, prop = 0.7, strata = subscribe)
train_data <- training(data_split)
test_data <- testing(data_split)

logistic_model <- logistic_reg() |> 
  set_engine("glm") |> 
  set_mode("classification")

logistic_recipe <- recipe(subscribe ~ gender_simple + experience, data = train_data)

logistic_workflow <- workflow() |> 
  add_model(logistic_model) |> 
  add_recipe(logistic_recipe)

fit_logistic <- fit(logistic_workflow, data = train_data)

# Predict and evaluate
predictions <- predict(fit_logistic, test_data, type = "prob") |> 
  bind_cols(predict(fit_logistic, test_data)) |> 
  bind_cols(test_data)

# Evaluate accuracy
metrics(predictions, truth = subscribe, estimate = .pred_class)

# Plot predicted probabilities
ggplot(predictions, aes(x = .pred_TRUE, fill = subscribe)) +
  geom_histogram(alpha = 0.6, position = "identity", bins = 30) +
  labs(
    title = "Predicted Probability of Subscribing",
    x = "Predicted Probability", 
    y = "Number of Players", 
    fill = "Actual Subscription"
  )


## Discussion

The KNN model demonstrated that behavioral metrics such as played hours, session frequency, and experience level are useful predictors of whether a player subscribes to the newsletter. Players with longer play times and more frequent sessions showed a higher likelihood of subscribing.

This aligns with expectations: more engaged users are more likely to seek updates and involvement with the game community. This analysis can guide the research team in targeting engaged users for promotional content and community building.

Some limitations include:

No session-level interaction or action data

Assumes static behavior over time

Uses basic features without engineered game event categories

Future work should explore logistic regression and decision trees to assess interpretability and feature impact. Social activity (e.g., simultaneous play) could also offer predictive power.